In [61]:
def gcd_ext(a, b):
    if a == 0:
        return b, 0, 1
    gcd, x1, y1 = gcd_ext(b % a, a)
    x = y1 - (b // a) * x1
    y = x1
    return gcd, x, y

def mod_inv(a, m):
    gcd, x, y = gcd_ext(a, m)
    if gcd != 1:
        return None  
    return (x % m + m) % m

In [62]:
import random

def miller_rabin(n, k=40):
    if n == 2 or n == 3: 
        return True
    if n % 2 == 0 or n < 2:     
        return False

    s, d = 0, n - 1
    while d % 2 == 0:
        s += 1
        d //= 2

    for _ in range(k):
        a = random.randrange(2, n - 1)
        x = pow(a, d, n)
        
        if x == 1 or x == n - 1:
            continue
        
        for _ in range(s - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False        
    return True

def random_prime(bits):
    while True:
        candidate = random.getrandbits(bits)
        candidate |= (1 << (bits - 1)) | 1
        
        primes_small = [3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43]
        if any(candidate % p == 0 for p in primes_small):
            continue

        if miller_rabin(candidate):
            return candidate


def GenerateKeyPair(bits=256):
    p = random_prime(bits)
    q = random_prime(bits)
    
    while p == q:
        q = random_prime(bits)
        
    n = p * q
    phi = (p - 1) * (q - 1)
    
    e = 2**16 + 1
     
    g, _, _ = gcd_ext(e, phi)
    while g != 1:
        e = random.randrange(3, phi, 2)
        g, _, _ = gcd_ext(e, phi)
        
    d = mod_inv(e, phi)
    
    return ((e, n), (d, p, q))

In [63]:
def Encrypt(message, public_key):
    e, n = public_key
    if message >= n:
        raise Exception("failure")
    return pow(message, e, n)

def Decrypt(ciphertext, private_key):
    d, p, q = private_key
    n = p * q
    return pow(ciphertext, d, n)

def Sign(message, private_key):
    d, p, q = private_key
    n = p * q
    return pow(message, d, n)

def Verify(message, sign, public_key):
    e, n = public_key
    check_val = pow(sign, e, n)
    return check_val == message

In [64]:
def SendKey(k, sender_priv, receiver_pub):
    S = Sign(k, sender_priv)
    k1 = Encrypt(k, receiver_pub)
    S1 = Encrypt(S, receiver_pub)
    return k1, S1

def ReceiveKey(k1, S1, receiver_priv, sender_pub):
    k = Decrypt(k1, receiver_priv)
    S = Decrypt(S1, receiver_priv)
    is_okay = Verify(k, S, sender_pub)
    return k, is_okay

In [65]:
def bold_green(text):
    return f"\033[1;32m{text}\033[0m"

def bold(text):
    return f"\033[1m{text}\033[0m"


print(bold("+------------------Генерація ключів-----------------+"))

print(bold_green("Генерація ключів Аліси"))
pub_A, priv_A = GenerateKeyPair(256)
n_A = pub_A[1]
print(bold("public Key (e, n):"), pub_A)
print(bold("private Key (d):"), priv_A[0])

print(bold_green("\nГенерація ключів Боба"))
attempts = 0
while True:
    attempts += 1
    pub_B, priv_B = GenerateKeyPair(256)
    n_B = pub_B[1]
    if n_B >= n_A:
        break
print(f"+--> Згенеровано з {attempts}-ї спроби (щоб n_B >= n_A)")
print(bold("public Key (e, n)"), pub_B)
print(bold("private Key (d)"), priv_B[0])


print(bold("\n\n+------------------Шифрування та дешифрування------------------+"))
# Аліса пише Бобу

M = "juicy"
M_int = int.from_bytes(M.encode('utf-8'), 'big')

print(bold("[Alice] Message:"), M)
print(bold("[Alice] Message Int:"), M_int)

# Шифрування
C = Encrypt(M_int, pub_B)
print(bold_green("\nАліса шифрує для Боба, використовуючи pub_B"))
print(bold("Ciphertext (C)"), C)

# Дешифрування
dec = Decrypt(C, priv_B)
text = dec.to_bytes((dec.bit_length() + 7) // 8, 'big').decode('utf-8')
print(bold_green("\nБоб дешифрує повідомлення, використовуючи priv_B"))
print(bold("[Bob] Decrypted:"), dec)
print(bold("[Bob] Decrypted text:"), text)



print(bold("\n\n+------------------Цифровий підпис------------------+"))
# Аліса підписує повідомлення

print(bold_green("Аліса створює підпис"))
sign = Sign(M_int, priv_A)
print(bold("sign (S):"), sign)

print(bold_green("\nБоб перевіряє підпис ключем Аліси"))
is_okay = Verify(M_int, sign, pub_A)

if is_okay:
    print("Check: Verify(S, pub_A) == M")
    print("successful")
else:
    print("failure")


print(bold("\n\n+------------------Key Exchange------------------+"))
# Аліса передає секретний ключ k Бобу

# Генерація сесійного ключа k
k = random.randrange(2**64, n_A - 1)
print(bold("[Alice] Генерує сесійний ключ k:"), k)

print(bold_green("1. Аліса надсилає пакет (SendKey)"))
# Аліса підписує k, потім шифрує (k, S)

k1, S1 = SendKey(k, priv_A, pub_B)
print("Дії: 1. S = k^d_A (mod n_A)  -> Підпис")
print("     2. k1 = k^e_B (mod n_B) -> Шифрування ключа")
print("     3. S1 = S^e_B (mod n_B) -> Шифрування підпису")
print(bold("Encrypted key (k1):"), k1)
print(bold("Encrypted sign (S1):"), S1)



print(bold_green("2. Боб отримує пакет (ReceiveKey)"))
# Боб розшифровує k1 та S1, потім перевіряє підпис

k_received, integ_valid = ReceiveKey(k1, S1, priv_B, pub_A)
print("Дії: 1. k = k1^d_B (mod n_B) -> Розшифровка ключа")
print("     2. S = S1^d_B (mod n_B) -> Розшифровка підпису")
print("     3. Verify(k, S, pub_A)  -> Перевірка підпису")

print(bold("Decrypted k:"), k_received)

confidentiality = (k == k_received)
integrity = integ_valid

def result(n, s):
    if s:
        print(f"{n}: successful")
    else:
        print(f"{n}: failure")

result(bold_green("Конфіденційність"), confidentiality)
result(bold_green("Цілісність"), integrity)




+------------------Генерація ключів-----------------+
Генерація ключів Аліси
public Key (e, n): (65537, 9233742091323722637214630514987744559697758492680735465620706462757039046947848671576950937358818621399178172652718020156498748182975162031068691308322893)
private Key (d): 3101349328077540666957115017408642953569236613529584037555014581964507736109552613782946745881464570522858354622116956548627218548317109615769305459284481

Генерація ключів Боба
+--> Згенеровано з 10-ї спроби (щоб n_B >= n_A)
public Key (e, n) (65537, 11048392443468060441190962613801373372183443847130591893455678385660640860168875262245904923148844862412179878981119185928287606232449295177891595957668807)
private Key (d) 9049678117060728329700353450287189282270769933897146702360411239767594824522240651538513646031841073311501454451304608655976393861598346429005732663219241


+------------------Шифрування та дешифрування------------------+
[Alice] Message: juicy
[Alice] Message Int: 457236374393

Аліса шифрує для Б